In [358]:
import pandas as pd
from pulp import *
from itertools import *
from mip import *
from partitions import *

In [343]:
BUDGET = 1000000

In [344]:
def get_data():
    df = pd.read_csv('all_wines_cleaned.csv').sort_values(['Pontuação','Pontos_Total'], ascending=False)
    #Incluir somente os vinhos disponíveis e com alguma pontuação, pois essas serão nossas variáveis de decisão principais
    data = df[df.Preço_Normal.notna() & df.Pontos_Total != 0].copy().reset_index(drop=True)
    #data.set_index('Nome', inplace=True)
    return data

In [373]:
data = get_data()

In [374]:
data = data.sort_values('Pontos_Total', ascending=False)

In [375]:
data.head(10)

,Nome,Link,País,Preço_Sócio,Preço_Normal,Pontuação,Avaliações,Somelier,Decantação,Olfativo,...,Vinícola,Tipo,Potencial_Guarda,Região,Preços_Cat,Pontuação_Cat,Estoque_Cat,Pontos_Total,Puro,Tipo_Cat
194,Bear Flag Red Blend 2018,https://wine.com.br/vinhos/bear-flag-red-blend...,Estados Unidos,39.90,46.94,4.0,1014.0,O Bear Flag foi feito com o esforço em comum d...,NaN,"Frutas negras, com um leve tostado e um toque ...",...,Bear Flag,Tinto,5.0,Califórnia,1.0,2,2,4056.0,0,NaN
195,Pedro Teixeira Colheita Selecionada Bairrada D...,https://wine.com.br/vinhos/pedro-teixeira-colh...,Portugal,34.90,41.06,4.0,389.0,"O renomado enólogo Osvaldo Amado fez, neste vi...",NaN,"Frutas vermelhas maduras, frutas em compota, e...",...,Adega de Cantanhede,Tinto,7.0,Bairrada,1.0,2,2,1556.0,0,NaN
196,Espumante Veuve D`Argent Blanc De Blancs Brut,https://wine.com.br/vinhos/espumante-veuve-d-a...,França,36.90,43.41,4.0,360.0,"Frutas brancas como pera e maçã, delicadas not...",NaN,"Frutas brancas como pera e maçã, delicadas not...",...,Veuve d'Argent,Espumante,2.0,Bourgogne,1.0,2,1,1440.0,0,Outros
197,Praça dos Marqueses Escolha I.G. Beira Atlânti...,https://wine.com.br/vinhos/praca-dos-marqueses...,Portugal,28.90,34.00,4.0,339.0,O nome Praça dos Marqueses é uma homenagem à p...,NaN,"Frutas vermelhas, frutas negras, floral",...,Adega de Cantanhede,Tinto,5.0,Beira Atlântico,1.0,2,2,1356.0,0,NaN
63,Clos de Los Siete By Michel Rolland 2016,https://wine.com.br/vinhos/clos-de-los-siete-b...,Argentina,78.90,92.82,4.5,268.0,Pontuado com altas notas por James Suckling e ...,20.0,"Amora, mirtilo, violeta, especiarias, amadeirado",...,Clos de los Siete,Tinto,10.0,Valle de Uco,2.0,2,2,1206.0,0,NaN
64,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,https://wine.com.br/vinhos/toro-loco-d-o-p-uti...,Espanha,24.90,29.29,4.5,261.0,"Mais do que um vinho, uma companhia para qualq...",NaN,"Frutas frescas, floral",...,BVC Bodegas,Tinto,4.0,Utiel,1.0,2,1,1174.5,0,NaN
198,Espumante Toro Loco D.O. Cava Brut,https://wine.com.br/vinhos/espumante-toro-loco...,Espanha,44.90,52.82,4.0,284.0,"Com aromas de flores brancas, frutas frescas c...",NaN,"Flores brancas, frutas frescas como maçã e per...",...,BVC Bodegas,Espumante,3.0,Utiel,1.0,2,1,1136.0,0,Outros
199,Viña Cosos D.O. Campo de Borja Garnacha Syrah ...,https://wine.com.br/vinhos/vina-cosos-d-o-camp...,Espanha,42.42,49.90,4.0,254.0,"Mesmo a vinícola sendo fundada em 1984, a Garn...",NaN,"Frutas vermelhas maduras, cerejas, ameixa e fl...",...,Bodegas Aragonesas,Tinto,5.0,Campo de Borja,1.0,2,2,1016.0,0,NaN
65,Casillero Reserva Limited Edition Cabernet Sau...,https://wine.com.br/vinhos/casillero-reserva-l...,NaN,44.90,52.82,4.5,203.0,"Edição especial para o Halloween, esse Caberne...",NaN,"Cassis, cereja, ameixa e notas de tostado",...,Concha y Toro,Tinto,4.0,NaN,1.0,2,1,913.5,1,NaN
66,Espumante Fantinel D.O.C. Prosecco Extra Dry,https://wine.com.br/vinhos/espumante-fantinel-...,Itália,54.90,64.59,4.5,187.0,"Elaborado pelo método Charmat, com ótimo fresc...",NaN,"Frutas cítricas, maçã verde, pêssego, floral",...,Fantinel,Espumante,3.0,Friuli,2.0,2,1,841.5,1,Outros


In [376]:
vinhos = data.index.to_list()

Os vinhos do catálogo possuem cerca de 15 por cento de desconto para os associados, i.e. `Preço_Normal ~ 0.85 * Preço_Sócio`

Considerando como uma aproximação que no Preço Normal incida aproximadamente 25% do valor do Preço de Custo do Vinho, i.e. 10% do Preço_Sócio.

In [377]:
custo = data['Preço_Normal'] / 1.25

In [378]:
data['Custo'] = custo

In [379]:
custo = dict(zip(vinhos, data.Custo))

In [380]:
preço = dict(zip(vinhos, data.Preço_Normal))

In [381]:
pontos = dict(zip(vinhos, data.Pontuação))

In [382]:
n_pontos = dict(zip(vinhos, data.Avaliações.to_list()))

A coluna adicional `Total_Pontos = Pontuação * Avaliações` será usada como otimizador do vinho

In [383]:
total_pontos = dict(zip(vinhos, data.Pontos_Total.to_list()))

In [384]:
def return_solution(prob):   
    return LpStatus[prob.status], {v:v.varValue for v in prob.variables() if v.varValue > 0}  

# Solução Trivial
Maximizar a pontuação total dentro do Orçamento

In [385]:
prob = LpProblem("Maximizar somente a pontuação total", LpMaximize)

## Variável

In [386]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, cat='Integer')

## Objetiva

In [387]:
prob += lpSum([total_pontos[i] * vinhos_var[i] for i in vinhos])

## Constraints
Orçamento

In [388]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

In [389]:
prob.solve()

1

In [390]:
status, solução = return_solution(prob)

In [391]:
solução

{Vinhos_194: 26629.0, Vinhos_197: 1.0}

In [392]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [393]:
list(solução.keys())

[194, 197]

In [394]:
solution = data.loc[solução.keys()].copy()

In [395]:
solution['Quantidade'] = solução.values()

In [396]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']]

,Quantidade,Nome,Custo,Pontuação,Avaliações
194,26629.0,Bear Flag Red Blend 2018,37.552,4.0,1014.0
197,1.0,Praça dos Marqueses Escolha I.G. Beira Atlânti...,27.200,4.0,339.0


A solução é trivial porque retorna o máximo possível de vinhos com a pontuação máxima, o vinho californiano `Bear Flag Red Blend` e com o valor restante compra 1 garrafa do vinho mais barato mas com boas pontuações.

Quantidade total do Estoque:

In [397]:
sum(solution.Quantidade.to_list())

26630.0

## Solução 2
Maximizar a pontuação total dentro do Orçamento mas limitar o número de garrafas à 5000.

In [398]:
prob = LpProblem("Limitar a quantidade à 5000", LpMaximize)

## Variável

In [399]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva

In [400]:
prob += lpSum([total_pontos[i] * vinhos_var[i] for i in vinhos])

## Constraints
Orçamento

In [401]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

In [402]:
prob.solve()

1

In [403]:
status, solução = return_solution(prob)

In [404]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [405]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {194: 5000.0, 195: 5000.0, 196: 5000.0, 197: 5000.0, 198: 4999.0, 199: 247.0, 200: 4.0, 64: 5000.0}


In [406]:
solution = data.loc[solução.keys()].copy()

In [407]:
solution['Quantidade'] = solução.values()

In [408]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
194,5000.0,Bear Flag Red Blend 2018,37.552,4.0,1014.0
195,5000.0,Pedro Teixeira Colheita Selecionada Bairrada D...,32.848,4.0,389.0
196,5000.0,Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,4.0,360.0
197,5000.0,Praça dos Marqueses Escolha I.G. Beira Atlânti...,27.200,4.0,339.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
198,4999.0,Espumante Toro Loco D.O. Cava Brut,42.256,4.0,284.0
199,247.0,Viña Cosos D.O. Campo de Borja Garnacha Syrah ...,39.920,4.0,254.0
200,4.0,Baron Philippe de Rothschild Mas Andes Caberne...,25.320,4.0,157.0


Quantidade de Garrafas no Estoque

In [409]:
sum(solution.Quantidade.to_list())

30250.0

A solução ainda é trivial porque há um catálogo muito pobre, seis vinhos somente com grande estoque e outros 2 com estoque baixo. No entanto a quantidade de garrafas é superior ao primeiro caso. Nada mais que ordenar pela quantidade de avaliações total.

## Solução 3
Maximizar a Pontuação total e com pontuação mínima 4

In [410]:
prob = LpProblem("Rastrear a Pontuação Mínima", LpMaximize)

## Variável
Número de Garrafas Limitada à 5000 para evitar a solução trivial

In [411]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva
Maximizar a pontuação total

In [412]:
prob += lpSum([total_pontos[i] * vinhos_var[i] for i in vinhos])

## Constraints
Orçamento

In [413]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

Pontuação Mínima deve ser 4

In [414]:
prob += lpSum([pontos[i] * vinhos_var[i] for i in vinhos]) >= sum([4 * vinhos_var[i] for i in vinhos]) 

In [415]:
prob.solve()

1

In [416]:
status, solução = return_solution(prob)

In [417]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [418]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {194: 5000.0, 195: 5000.0, 196: 5000.0, 197: 5000.0, 198: 4999.0, 199: 247.0, 200: 4.0, 64: 5000.0}


In [419]:
solution = data.loc[solução.keys()].copy()

In [420]:
solution['Quantidade'] = solução.values()

In [421]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
194,5000.0,Bear Flag Red Blend 2018,37.552,4.0,1014.0
195,5000.0,Pedro Teixeira Colheita Selecionada Bairrada D...,32.848,4.0,389.0
196,5000.0,Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,4.0,360.0
197,5000.0,Praça dos Marqueses Escolha I.G. Beira Atlânti...,27.200,4.0,339.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
198,4999.0,Espumante Toro Loco D.O. Cava Brut,42.256,4.0,284.0
199,247.0,Viña Cosos D.O. Campo de Borja Garnacha Syrah ...,39.920,4.0,254.0
200,4.0,Baron Philippe de Rothschild Mas Andes Caberne...,25.320,4.0,157.0


A solução encontrada é idêntica à solução 2 tendo em vista que já obedecia ao Constraint adicional.

## Solução 4

In [422]:
prob = LpProblem("O preço como variável objetiva", LpMaximize)

## Variável
Número de Garrafas Limitada à 5000 para evitar a solução trivial

In [423]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva
Maximizar a pontuação total

In [424]:
prob += lpSum([vinhos_var[i] * 1./preço[i] for i in vinhos])

## Constraints
Orçamento

In [425]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

Para a Pontuação Mínima ser 4, devemos majorar a pontuação média por um valor maior

In [426]:
prob += lpSum([pontos[i] * vinhos_var[i] for i in vinhos]) >= sum([4.5 * vinhos_var[i] for i in vinhos])  

In [427]:
prob.solve()

1

In [428]:
status, solução = return_solution(prob)

In [429]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [430]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {121: 5000.0, 151: 5000.0, 160: 5000.0, 216: 5000.0, 295: 5000.0, 56: 5000.0, 62: 5000.0, 64: 5000.0, 74: 3404.0}


In [431]:
solution = data.loc[solução.keys()].copy()

In [432]:
solution['Quantidade'] = solução.values()

In [433]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
121,5000.0,Dark Horse Rosé Bubbles Lata 375ml,23.432,4.5,14.0
151,5000.0,Dark Horse Pinot Grigio Lata 375ml,23.432,4.5,8.0
160,5000.0,Dark Horse Rosé Lata 375ml,23.432,4.5,6.0
216,5000.0,Terralis Syrah Malbec 2018,20.608,4.0,55.0
295,5000.0,Espumante Baby Chandon Réserve Brut 187 ml.,22.496,4.0,14.0
56,5000.0,Frisante Miolo Moscatel Blanc,22.496,5.0,1.0
62,5000.0,Viña Carrasco Rosé 2018,23.432,5.0,1.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
74,3404.0,Canepa Novísimo Merlot 2018,25.320,4.5,72.0


In [7]:
#data.index = data.index.str.replace(" ", "_")

Quantidade de Garrafas no Estoque

In [434]:
sum(solution.Quantidade.to_list())

43404.0

Está claro que se não embutirmos constraints mais sofisticados somente temos soluções triviais, encontradas facilmente simplesmente ordenando  DataFrame

## Solução 4
* Minimizar o Custo como variável objetiva
* Maximixar a Pontuação
* Maximizar o nº de Avaliações

In [435]:
prob = LpProblem("Maximo Pontuação, mínimo Preço", LpMaximize)

## Variável
Número de Garrafas Limitada à 5000 para evitar a solução trivial

In [436]:
vinhos_var = LpVariable.dicts("Vinhos", vinhos, lowBound=0, upBound=5000, cat='Integer')

## Objetiva
Maximizar a pontuação total

In [437]:
prob += lpSum([vinhos_var[i] * 1./preço[i] for i in vinhos])

## Constraints
Orçamento

In [438]:
prob += lpSum([custo[i] * vinhos_var[i] for i in vinhos]) <= BUDGET

Pontuação Mínima deve ser 4

In [439]:
prob += lpSum([pontos[i] * vinhos_var[i] for i in vinhos]) >= 4.5 * sum([vinhos_var[i] for i in vinhos]) 

In [440]:
prob.solve()

1

In [441]:
status, solução = return_solution(prob)

In [442]:
solução = {int(str(k).split('Vinhos_')[1]):v for k,v in solução.items()}

In [443]:
print(f'Status: {status}')
print(f'Solução Encontrada: {solução}')

Status: Optimal
Solução Encontrada: {121: 5000.0, 151: 5000.0, 160: 5000.0, 216: 5000.0, 295: 5000.0, 56: 5000.0, 62: 5000.0, 64: 5000.0, 74: 3404.0}


In [444]:
solution = data.loc[solução.keys()].copy()

In [445]:
solution['Quantidade'] = solução.values()

In [446]:
solution[["Quantidade", 'Nome', 'Custo', 'Pontuação', 'Avaliações']].sort_values("Quantidade",ascending=False)

,Quantidade,Nome,Custo,Pontuação,Avaliações
121,5000.0,Dark Horse Rosé Bubbles Lata 375ml,23.432,4.5,14.0
151,5000.0,Dark Horse Pinot Grigio Lata 375ml,23.432,4.5,8.0
160,5000.0,Dark Horse Rosé Lata 375ml,23.432,4.5,6.0
216,5000.0,Terralis Syrah Malbec 2018,20.608,4.0,55.0
295,5000.0,Espumante Baby Chandon Réserve Brut 187 ml.,22.496,4.0,14.0
56,5000.0,Frisante Miolo Moscatel Blanc,22.496,5.0,1.0
62,5000.0,Viña Carrasco Rosé 2018,23.432,5.0,1.0
64,5000.0,Toro Loco D.O.P. Utiel-Requena Tinto Superior ...,23.432,4.5,261.0
74,3404.0,Canepa Novísimo Merlot 2018,25.320,4.5,72.0


Quantidade de Garrafas no Estoque

In [447]:
sum(solution.Quantidade.to_list())

43404.0

In [448]:
sum(row.Quantidade * row.Custo for row in solution.itertuples()) <= BUDGET

True

Está claro que se não embutirmos constraints mais sofisticados somente temos soluções triviais, encontradas facilmente simplesmente ordenando  DataFrame

## Estratégia
Os dados de negócio da wine.com são:
* 100.000 clientes
* 2 vinhos distintos por mês
* 569 vinhos disponíveis
* 3780 vinhos distintos no catálogo

Mock-up de Negócios: 1% do escopo da wine com Orçamento de R$ 1.000.000
* Projeção de 1000 Clientes
* 2 vinhos por mês
* Estoque para 6 meses

Isso resulta inicialmente no mínimo de 12000 garrafas no estoque, 12 vinhos distintos. 
Vamos começar com essas condições iniciais.

Como devemos ter no mínimo 1000 garrafas de cada vinho escolhido, vamos modelar variáveis binárias indicando se um vinho foi escolhido para a compra. E essa compra será de 1000 vinhos.

In [449]:
data.set_index('Nome', inplace=True)

In [450]:
vinhos = data.index.to_list()
n = range(len(vinhos))
custo = [float(i) for i in data.Custo.to_list()]
pontos = [float(i) for i in data.Pontuação.to_list()]
avaliação = [int(i) for i in data.Avaliações.to_list()]
total_pontos = [int(i) for i in data.Pontos_Total.to_list()]
guarda = []
for i in data.Potencial_Guarda:
    if not bool(i):
        print(i)
    else:
        guarda.append(i)
    NUM_WINE = 1000
SOLUTIONS = {}

In [451]:
data.Potencial_Guarda.isna().sum()

0

In [452]:
def return_solution(model):
    model.max_gap = 0.05
    status = model.optimize(max_seconds=300)
    if status == OptimizationStatus.OPTIMAL:
        print('optimal solution cost {} found'.format(model.objective_value))
    elif status == OptimizationStatus.FEASIBLE:
        print('sol.cost {} found, best possible: {}'.format(model.objective_value, model.objective_bound))
    elif status == OptimizationStatus.NO_SOLUTION_FOUND:
        print('no feasible solution found, lower bound is: {}'.format(model.objective_bound))
    if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
        return {v.name:v.x for v in m.vars if abs(v.x) > 1e-6}
    return None

In [524]:
def safra_stats(df):
    print(f'Nº de Vinhos Distintos: {df.shape[0]}')
    print(f'Mediana do Preço: {df.Preço_Normal.median()}')
    print(f'Mediana da Pontuação: {df.Pontuação.median()}')
    print(f'Mediana do nº de Avaliações: {df.Avaliações. median()}')
    print(f'Total do Orçamento Utilizado: {df.Custo.sum() * 1000:.2f}')
    #print(f'Valor Bruto da Venda Avulsa: {df.Preço_Normal.sum() * 1000:.2f}')
    #print(f'Valor Bruto da Venda aos Associados: {df.Preço_Sócio.sum() * 1000:.2f}')
    print(f'Distribuição da Pontuação: \n{df.Pontuação.value_counts(ascending=False)}')    
    print(f'Distribuição do Potencial de Guarda: \n{df.Potencial_Guarda.value_counts(ascending=False)}')    

## Iteração I
Maximiza Pontos Totais dentro do Orçamento com Avaliação no mínimo 4

In [454]:
name = "Maximiza Pontos Totais dentro do Orçamento com Avaliação no mínimo 4"
m = Model(name) # use GRB for Gurobi
wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in vinhos]

Um modo de maximizar tanto a pontuação quanto o número de avaliações é maximizar a contagem total de pontos e ao mesmo tempo colocar um constraint na pontuação mínima.

In [455]:
m.objective = maximize(xsum(wines[i] * total_pontos[i] for i in n))
m += xsum(wines[i] * custo[i] for i in n) <= BUDGET / NUM_WINE
for i in n:
    m += wines[i] * pontos[i] >= 4 * wines[i]

Relaxação básica do orçamento, se tivermos um valor menor no orçamento do que o vinho mais barato não podemos comprar a safra

In [456]:
m += xsum(wines[i] * custo[i] for i in n) >= BUDGET / NUM_WINE - data.Custo.min() + 1

In [457]:
solução = return_solution(m)

optimal solution cost 20817.0 found


In [458]:
SOLUTIONS[name] = data.loc[solução.keys()].copy()
df = SOLUTIONS[name]

In [459]:
df[['Custo', 'Preço_Normal', 'Preço_Sócio', 'Pontuação', 'Avaliações', 'Pontos_Total', 'Potencial_Guarda']].sort_values("Potencial_Guarda")

,Custo,Preço_Normal,Preço_Sócio,Pontuação,Avaliações,Pontos_Total,Potencial_Guarda
Nome,,,,,,,
Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,43.41,36.90,4.0,360.0,1440.0,2.0
Espumante Salton Moscatel,30.968,38.71,32.90,4.5,72.0,324.0,2.0
Palafitos de Mar Semi Sweet Rosé Cabernet Sauvignon/Syrah 2019,29.080,36.35,30.90,4.0,47.0,188.0,3.0
Canepa Novísimo Syrah 2018,25.320,31.65,26.90,4.0,53.0,212.0,3.0
Espumante Salton Brut,30.968,38.71,32.90,4.5,60.0,270.0,3.0
Espumante Toro Loco D.O. Cava Brut,42.256,52.82,44.90,4.0,284.0,1136.0,3.0
Espumante Fantinel D.O.C. Prosecco Extra Dry,51.672,64.59,54.90,4.5,187.0,841.5,3.0
Canepa Novísimo Merlot 2018,25.320,31.65,26.90,4.5,72.0,324.0,3.0
Espumante Louis Bouillot Rosé Brut,61.080,76.35,64.90,4.5,109.0,490.5,3.0


In [460]:
safra_stats(df)

Nº de Vinhos Distintos: 29
Mediana da Pontuação: 4.0
Distribuição da Pontuação: 
4.0    18
4.5    11
Name: Pontuação, dtype: int64
Mediana do nº de Avaliações: 99.0
Total do Orçamento Utilizado: 999168.00
Distribuição do Potencial de Guarda: 
4.0     10
5.0      8
3.0      7
2.0      2
10.0     1
7.0      1
Name: Potencial_Guarda, dtype: int64


A solução encontrada atende aos requisitos impostos para 14 meses, no entanto temos vários pontos de melhoria:
* Vinhos com potencial de guarda baixo, i.e. de 2 e 3 anos
 * Aumentar o Constraint para o Potencial de Guarda
* Utilizar o máximo possível do Orçamento

## Iteração II
Maximiza o número de Avaliações com Pontuação no mínimo 4 dentro do Orçamento e com `constraint` rígido no Orçamento

In [467]:
name = "Maximiza o número de Avaliações com Pontuação no mínimo 4 dentro do Orçamento"
m = Model(name) # use GRB for Gurobi
wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in vinhos]

Trocar a pontuação total pelo número de Avaliações

In [468]:
m.objective = maximize(xsum(wines[i] * avaliação[i] for i in n))

Colocar um limitante inferior *constraint* mais rígido ao Orçamento

In [469]:
m += xsum(wines[i] * custo[i] for i in n) <= BUDGET / NUM_WINE

Relaxação básica do orçamento, se tivermos um valor menor no orçamento do que o vinho mais barato não podemos comprar a safra

In [470]:
m += xsum(wines[i] * custo[i] for i in n) >= (BUDGET / NUM_WINE) - data.Custo.min() + 1e-6

Aumentar o *constraint* da pontuação

In [471]:
for i in n:
    m += wines[i] * pontos[i] >= 4 * wines[i]

In [472]:
solução = return_solution(m)

optimal solution cost 5042.0 found


In [473]:
SOLUTIONS[name] = data.loc[solução.keys()].copy()
df = SOLUTIONS[name]

In [474]:
df[['Custo', 'Preço_Normal', 'Preço_Sócio', 'Pontuação', 'Avaliações', 'Pontos_Total', 'Potencial_Guarda']].sort_values("Potencial_Guarda")

,Custo,Preço_Normal,Preço_Sócio,Pontuação,Avaliações,Pontos_Total,Potencial_Guarda
Nome,,,,,,,
Espumante Veuve D`Argent Blanc De Blancs Brut,34.728,43.41,36.90,4.0,360.0,1440.0,2.0
Espumante Salton Moscatel,30.968,38.71,32.90,4.5,72.0,324.0,2.0
Espumante Salton Brut,30.968,38.71,32.90,4.5,60.0,270.0,3.0
Canepa Novísimo Merlot 2018,25.320,31.65,26.90,4.5,72.0,324.0,3.0
Espumante Toro Loco D.O. Cava Brut,42.256,52.82,44.90,4.0,284.0,1136.0,3.0
Canepa Novísimo Syrah 2018,25.320,31.65,26.90,4.0,53.0,212.0,3.0
Espumante Louis Bouillot Rosé Brut,61.080,76.35,64.90,4.5,109.0,490.5,3.0
Espumante Fantinel D.O.C. Prosecco Extra Dry,51.672,64.59,54.90,4.5,187.0,841.5,3.0
Dancing Flame Ojos del Salado Merlot 2018,24.376,30.47,25.90,4.0,70.0,280.0,4.0


In [475]:
safra_stats(df)

Nº de Vinhos Distintos: 29
Mediana da Pontuação: 4.0
Distribuição da Pontuação: 
4.0    20
4.5     9
Name: Pontuação, dtype: int64
Mediana do nº de Avaliações: 99.0
Total do Orçamento Utilizado: 995416.00
Distribuição do Potencial de Guarda: 
4.0     10
5.0      8
3.0      6
2.0      2
6.0      1
10.0     1
7.0      1
Name: Potencial_Guarda, dtype: int64


Conseguimos alocar quase integralmente o orçamento com uma solução ótima, com pontuação mínima 4 e alto número de avaliações médio, no entanto ainda temos alguns vinhos com poucas avaliações, vamos incluir mais um constraint rígido ao potencial de guarda e número de avaliações.

## Iteração III
Maximiza o número de Avaliações
 * Pontuação no mínimo 4
 * Número de Avaliações mínimas 10
 * Potencial de Guarda mínimo 5
 * Utilização máxima do Orçamento

In [526]:
name = "Maximiza Avaliações, Pontuação mínima 4, Avaliações mínimas 10, Potencial de Guarda 5 e utilização máxima do Orçamento"
m = Model(name) # use GRB for Gurobi
wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in vinhos]

Trocar a pontuação total pelo número de Avaliações

In [527]:
m.objective = maximize(xsum(wines[i] * avaliação[i] for i in n))

Colocar um limitante inferior *constraint* mais rígido ao Orçamento

In [528]:
m += xsum(wines[i] * custo[i] for i in n) <= BUDGET / NUM_WINE

Relaxação básica do orçamento, se tivermos um valor menor no orçamento do que o vinho mais barato não podemos comprar a safra

In [529]:
m += xsum(wines[i] * custo[i] for i in n) >= (BUDGET / NUM_WINE)

Pontuação Mínima 4 para todos os vinhos

In [530]:
for i in n:
    m += wines[i] * pontos[i] >= 4 * wines[i]

Numero de Avaliações

In [531]:
for i in n:
    m += wines[i] * avaliação[i] >= wines[i] * 10

Potencial Guarda no Mínimo 5

In [532]:
for i in n:
    m += wines[i] * guarda[i] >= wines[i] * 5

In [533]:
solução = return_solution(m)

optimal solution cost -3357.0 found


In [534]:
SOLUTIONS[name] = data.loc[solução.keys()].copy()
df = SOLUTIONS[name]

In [535]:
df[['Custo', 'Preço_Normal', 'Preço_Sócio', 'Pontuação', 'Avaliações', 'Pontos_Total', 'Potencial_Guarda']].sort_values("Potencial_Guarda")

,Custo,Preço_Normal,Preço_Sócio,Pontuação,Avaliações,Pontos_Total,Potencial_Guarda
Nome,,,,,,,
Bear Flag Red Blend 2018,37.552,46.94,39.90,4.0,1014.0,4056.0,5.0
Portas da Herdade Tinto 2018,27.200,34.00,28.90,4.0,53.0,212.0,5.0
Genio Español Roble D.O.P. Jumilla Monastrell 2016,41.320,51.65,43.90,4.0,56.0,224.0,5.0
Nederburg 56 Hundred Pinotage 2018,25.320,31.65,26.90,4.0,57.0,228.0,5.0
Pájaro de Buen Agüero D.O.P. Cariñena G 2018,78.616,98.27,83.53,4.0,60.0,240.0,5.0
Que Guapo Malbec Blend 2017,37.552,46.94,39.90,4.0,63.0,252.0,5.0
Root: 1 Reserva Heritage Red 2018,33.792,42.24,35.90,4.5,56.0,252.0,5.0
La Mora D.O.C. Maremma Toscana Rosso 2015,46.968,58.71,49.90,4.5,60.0,270.0,5.0
Finca Traversa Tannat 2018,24.376,30.47,25.90,4.0,37.0,148.0,5.0


In [536]:
safra_stats(df)

Nº de Vinhos Distintos: 22
Mediana do Preço: 50.775
Mediana da Pontuação: 4.0
Mediana do nº de Avaliações: 65.0
Total do Orçamento Utilizado: 1000000.00
Distribuição da Pontuação: 
4.0    14
4.5     8
Name: Pontuação, dtype: int64
Distribuição do Potencial de Guarda: 
5.0     16
6.0      3
8.0      1
10.0     1
7.0      1
Name: Potencial_Guarda, dtype: int64


Conseguimos alocar **integralmente** o orçamento, com uma solução ótima, o que é mais impressionante, com 22 vinhos distintos, i.e. uma safra para 11 meses, com pontuação mínima 4, Número de Avaliações Mínimas 10 e potencial de guarda no mínimo 5 para os vinhos disponíveis

## Iteração IV
Minimiza o preço
 * Pontuação no mínimo 4
 * Número de Avaliações mínimas 10
 * Potencial de Guarda mínimo 5
 * Utilização máxima do Orçamento

In [511]:
name = "Minimiza o Custo, Pontuação mínima 4, Avaliações mínimas 10, Potencial de Guarda 5 e utilização máxima do Orçamento"
m = Model(name) # use GRB for Gurobi
wines = [m.add_var(name=vinho, var_type=BINARY) for vinho in vinhos]

Função Objetiva de Minimizar o Custo

In [512]:
m.objective = maximize(xsum(wines[i] * 1./custo[i] for i in n))

Colocar um limitante inferior *constraint* mais rígido ao Orçamento

In [513]:
m += xsum(wines[i] * custo[i] for i in n) <= BUDGET / NUM_WINE

Relaxação básica do orçamento, se tivermos um valor menor no orçamento do que o vinho mais barato não podemos comprar a safra

In [514]:
m += xsum(wines[i] * custo[i] for i in n) >= (BUDGET / NUM_WINE) - data.Custo.min()

Pontuação Mínima 4 para todos os vinhos

In [515]:
for i in n:
    m += wines[i] * pontos[i] >= 4 * wines[i]

Numero de Avaliações

In [516]:
for i in n:
    m += wines[i] * avaliação[i] >= wines[i] * 10

Potencial Guarda no Mínimo 5

In [517]:
for i in n:
    m += wines[i] * guarda[i] >= wines[i] * 5

In [518]:
solução = return_solution(m)

optimal solution cost 0.8207418726019119 found


In [519]:
SOLUTIONS[name] = data.loc[solução.keys()].copy()
df = SOLUTIONS[name]

In [520]:
df[['Custo', 'Preço_Normal', 'Preço_Sócio', 'Pontuação', 'Avaliações', 'Pontos_Total', 'Potencial_Guarda']].sort_values("Potencial_Guarda")

,Custo,Preço_Normal,Preço_Sócio,Pontuação,Avaliações,Pontos_Total,Potencial_Guarda
Nome,,,,,,,
Bear Flag Red Blend 2018,37.552,46.94,39.90,4.0,1014.0,4056.0,5.0
Altivo Vineyard Selection Cabernet Sauvignon 2017,40.376,50.47,42.90,4.0,16.0,64.0,5.0
Infame Reserva Cabernet Sauvignon 2017,43.200,54.00,45.90,4.0,16.0,64.0,5.0
Señorío de Ayud Cabernet Sauvignon Syrah 2018,27.200,34.00,28.90,4.5,15.0,67.5,5.0
Casillero del Diablo Cabernet Sauvignon/Merlot 2018,45.080,56.35,47.90,4.5,16.0,72.0,5.0
Señorío De Ayud Garnacha Syrah 2018,30.968,38.71,32.90,4.0,18.0,72.0,5.0
Vale da Coruja RL Tinto 2016,33.792,42.24,35.90,4.0,18.0,72.0,5.0
La Chamiza Malbec 2018,30.024,37.53,31.90,4.0,25.0,100.0,5.0
Bobal deSanjuan Viñas Viejas D.O.P. Utiel-Requena 2016,44.144,55.18,46.90,4.0,27.0,108.0,5.0


In [525]:
safra_stats(df)

Nº de Vinhos Distintos: 28
Mediana do Preço: 42.825
Mediana da Pontuação: 4.0
Mediana do nº de Avaliações: 37.5
Total do Orçamento Utilizado: 985088.00
Distribuição da Pontuação: 
4.0    24
4.5     4
Name: Pontuação, dtype: int64
Distribuição do Potencial de Guarda: 
5.0    24
6.0     3
7.0     1
Name: Potencial_Guarda, dtype: int64


Conseguimos alocar o máximo possível dentro do orçamento com uma solução ótima, com 28 vinhos distintos, i.e. uma safra para 14 meses, com pontuação mínima 4, Número de Avaliações Mínimas 10 e potencial de guarda no mínimo 5 para os vinhos disponíveis. Como tentamos minimizar o Preço, o preço médio cai. O número de avaliações também diminui drasticamente do modelo anterior, significando que não necessariamente que os vinhos mais baratos são os mais populares. Os vinhos mais populares são aqueles que são baratos _o suficiente_ e bons claro porque não basta ser barato.